In [1]:
import requests
import json
import io
import numpy as np
import re

## Insert data

In [19]:
ts1 = '{"1": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'
ts2 = '{"2": {"ts": {"times": [1, 2, 3], "values": [1, 5, 10]}}}'

In [20]:
r = requests.post('http://127.0.0.1:5000/insert', json = ts1)
r = requests.post('http://127.0.0.1:5000/insert', json = ts2)

## Request data

In [4]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))

In [5]:
r.text

'<pre>{\n    "2": {},\n    "1": {}\n}</pre>'

In [7]:
assert r.text == '<pre>{\n    "1": {},\n    "2": {}\n}</pre>' or '<pre>{\n    "2": {},\n    "1": {}\n}</pre>'

In [925]:
r = requests.get('http://127.0.0.1:5000/select?fields=ts', auth=('user', 'pass'))
# r.json()

## Insert meta

In [21]:
meta = {'1': {'order': 1, 'blarg': 1}}
meta2 = {'2': {'order': 2, 'blarg': 2}}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
# print(meta)
s_meta = io.StringIO()
json.dump(meta, fp=s_meta)
s_meta.getvalue()

s_meta2 = io.StringIO()
json.dump(meta2, fp=s_meta2)
s_meta2.getvalue()

'{"2": {"blarg": 2, "order": 2}}'

In [22]:
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = s_meta.getvalue())
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = s_meta2.getvalue())

In [10]:
r = requests.get('http://127.0.0.1:5000/select?fields=order', auth=('user', 'pass'))
r.text

'<pre>{\n    "2": {\n        "order": 2\n    },\n    "1": {\n        "order": 1\n    }\n}</pre>'

In [31]:
assert r.json() == {'1': {'order': 1}, '2': {'order': 2}}

## Sort by order

In [12]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border', auth=('user', 'pass'))
r.text

'<pre>{\n    "1": {},\n    "2": {}\n}</pre>'

In [399]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border,limit,1', auth=('user', 'pass'))
r.json()

{'10': {'blarg': 1, 'order': 1, 'pk': '10'}}

In [53]:
jsonfile = r.json()
jsonfile

{'10': {'ts': {'times': [1, 2, 3], 'values': [4, 5, 6]}}}

In [194]:
s = io.StringIO()
json.dump(jsonfile, fp=s)
s.getvalue()

'{"10": {"ts": {"values": [4, 5, 6], "times": [1, 2, 3]}}}'

In [260]:
other_value = '{"10": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'

In [267]:
r = requests.post('http://127.0.0.1:5000/insert', json = s.getvalue())

In [81]:
s.getvalue()

'{"10": {"ts": {"values": [4, 5, 6], "times": [1, 2, 3]}}}'

In [86]:
data = json.loads(s.getvalue())

In [643]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))
r.json()

{'10': {'pk': '10'}, '12': {'pk': '12'}}

## Filter by meta

In [689]:
r = requests.get('http://127.0.0.1:5000/select?md=blarg,<2', auth=('user', 'pass'))
# {'blarg': {'<=': 2}}
r.json()

{}

## Select augmented

In [13]:
augmented = {'md': {"blarg": {'>=': 1}},'proc':'stats','target':['damean','dastd'], 'additional':{"sort_by": "+order", "limit": 2}}
augemented_json = io.StringIO()
json.dump(augmented, fp=augemented_json)
augmented_data = augemented_json.getvalue()
augmented_data

'{"md": {"blarg": {">=": 1}}, "target": ["damean", "dastd"], "proc": "stats", "additional": {"limit": 2, "sort_by": "+order"}}'

In [34]:
r = requests.post('http://127.0.0.1:5000/augmented_select', json = augmented_data)
print (r.text)
# r.text

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>TimeoutError // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not by accident trigger a request to /favicon.ico which might
         change the application state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 4402122704,
          CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "ltdYXzEBL2aUGJ2pBjxy";
    </script>
  </head>
  <body>
    <div class="debugger">
<h1>

## Delete ts

In [86]:
r = requests.post('http://127.0.0.1:5000/delete', json = '2')

## Find similar

In [38]:
ts1 = '{{"times": [1, 3, 3], "values": [2, 5, 9]}}'
r = requests.post('http://127.0.0.1:5000/find_similar', json = ts1)

In [41]:
ts = {'times': [1,3,4],'values':[2,5,9]}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
# print(meta)
ts_a = io.StringIO()
json.dump(ts, fp=ts)
ts.getvalue()

meta = {'1': {'order': 1, 'blarg': 1}}
meta2 = {'2': {'order': 2, 'blarg': 2}}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
# print(meta)
s_meta = io.StringIO()
json.dump(meta, fp=s_meta)
s_meta.getvalue()

TypeError: <_io.StringIO object at 0x106440a68> is not JSON serializable